#Entregable Grupo 8 Aprendizaje Supervisado Integrantes
**XGBoost con ajuste de hiperparametros por GridSearchCV**

* Abdala, Agustin
* Acosta, Martín
* Baulina, David
* Giovine, Carina


In [ ]:
# sklearn version fixed to avoid known skopt issue
# con import de sklearn.metrics: 'from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, classification_report'
!pip install scikit-optimize scikit-learn==0.20.3 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100 kB 6.1 MB/s 
     |████████████████████████████████| 5.4 MB 54.9 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.20.3 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.20.3 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
# from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, classification_report
from sklearn.tree import DecisionTreeClassifier
from skopt import BayesSearchCV
import missingno as msno


# sklearn
from sklearn.tree import DecisionTreeClassifier

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/gradient_boosting.py:34: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ._gradient_boosting import predict_stages
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/gradient_boosting.py:34: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ._gradient_boosting import predict_stages


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/DiploDatos/AprendizajeSupervisado/master/Pr%C3%A1ctico/train.csv')
df.drop(['PassengerId','Name'], axis=1, inplace=True)

In [ ]:
df_imputed = df.copy()
categorics = ['HomePlanet', 'CryoSleep', 'Cabin', 'VIP', 'Destination' ]
categoric_imputer = SimpleImputer(strategy='most_frequent')
df_imputed[categorics] = categoric_imputer.fit_transform(df_imputed[categorics])
df_imputed.isna().sum()

/usr/local/lib/python3.7/dist-packages/sklearn/impute.py:341: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  row_mask = np.logical_not(row_mask).astype(np.bool)
/usr/local/lib/python3.7/dist-packages/sklearn/impute.py:341: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  row_mask = np.logical_not(row_mask).astype(np.bool)
/usr/local/lib/python3.7/dist-

HomePlanet        0
CryoSleep         0
Cabin             0
Destination       0
Age             162
VIP               0
RoomService     154
FoodCourt       155
ShoppingMall    188
Spa             158
VRDeck          169
Transported       0
dtype: int64

In [ ]:
imputer_cols = ["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck" ,"RoomService"]
df_2 = df_imputed.copy()
imputer = SimpleImputer(strategy="median")
df_2[imputer_cols] = imputer.fit_transform(df_2[imputer_cols])

In [ ]:
df_3 = df_2.copy()
df_3[['Deck', 'Num', 'Side']] = df_3['Cabin'].str.split('/', expand=True)
df_3.drop(columns = 'Cabin', inplace = True)
df_3.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Num,Side
0,Europa,False,TRAPPIST-1e,57.0,False,0.0,3859.0,13.0,195.0,3874.0,0,C,123,P
1,Mars,True,TRAPPIST-1e,18.0,False,0.0,0.0,0.0,0.0,0.0,1,F,126,S
2,Europa,True,TRAPPIST-1e,33.0,False,0.0,0.0,0.0,0.0,0.0,1,B,343,S
3,Earth,False,TRAPPIST-1e,43.0,False,142.0,1208.0,0.0,0.0,204.0,0,F,1420,P
4,Earth,False,TRAPPIST-1e,19.0,False,0.0,0.0,670.0,1.0,34.0,0,G,9,P


In [ ]:
categorical_cols_2 = ['HomePlanet', 'CryoSleep', 'VIP', 'Destination', 'Deck', 'Side']

for col in categorical_cols_2:
    df_3[col] = df_3[col].astype(str)
    df_3[col] = LabelEncoder().fit_transform(df_3[col])
df_3.describe()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Side
count,7823.00000,7823.000000,7823.000000,7823.000000,7823.000000,7823.000000,7823.000000,7823.000000,7823.000000,7823.000000,7823.000000,7823.000000,7823.000000
mean,0.65512,0.352295,1.492266,28.840470,0.023393,221.627764,446.615237,168.452512,298.862713,289.093570,0.506711,4.336188,0.514253
std,0.79611,0.477716,0.815762,14.313512,0.151156,673.194124,1606.401684,591.285786,1122.738262,1082.461294,0.499987,1.780309,0.499829
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.00000,0.000000,1.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000
50%,0.00000,0.000000,2.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,5.000000,1.000000
75%,1.00000,1.000000,2.000000,37.000000,0.000000,38.500000,56.500000,23.000000,50.000000,40.000000,1.000000,6.000000,1.000000
max,2.00000,1.000000,2.000000,79.000000,1.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000,1.000000,7.000000,1.000000


In [ ]:
y = df.Transported
y

0       0
1       1
2       1
3       0
4       0
       ..
7818    1
7819    0
7820    1
7821    0
7822    1
Name: Transported, Length: 7823, dtype: int64

In [ ]:
X = df_3.drop(columns = 'Transported')
X.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Num,Side
0,1,0,2,57.0,0,0.0,3859.0,13.0,195.0,3874.0,2,123,0
1,2,1,2,18.0,0,0.0,0.0,0.0,0.0,0.0,5,126,1
2,1,1,2,33.0,0,0.0,0.0,0.0,0.0,0.0,1,343,1
3,0,0,2,43.0,0,142.0,1208.0,0.0,0.0,204.0,5,1420,0
4,0,0,2,19.0,0,0.0,0.0,670.0,1.0,34.0,6,9,0


In [ ]:
x_names = X.columns
X = StandardScaler().fit_transform(X)
X = pd.DataFrame(X, columns=x_names)
X.head()

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Num,Side
0,0.433235,-0.737504,0.622444,1.967465,-0.154767,-0.329239,2.124377,-0.262923,-0.092514,3.312022,-1.312321,-0.945999,-1.028924
1,1.689423,1.355926,0.622444,-0.757408,-0.154767,-0.329239,-0.278040,-0.284910,-0.266208,-0.267088,0.372887,-0.940067,0.971889
2,0.433235,1.355926,0.622444,0.290620,-0.154767,-0.329239,-0.278040,-0.284910,-0.266208,-0.267088,-1.874058,-0.510990,0.971889
3,-0.822954,-0.737504,0.622444,0.989305,-0.154767,-0.118291,0.473999,-0.284910,-0.266208,-0.078616,0.372887,1.618572,-1.028924
4,-0.822954,-0.737504,0.622444,-0.687539,-0.154767,-0.329239,-0.278040,0.848286,-0.265317,-0.235676,0.934623,-1.171412,-1.028924


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state = 8)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [ ]:
import xgboost as xgb
from copy import deepcopy
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,confusion_matrix,roc_curve

results_dict = {}
xgbc0 = xgb.XGBClassifier(objective='binary:logistic',
                          booster='gbtree',
                          eval_metric='auc',
                          tree_method='hist',
                          grow_policy='lossguide',
                          use_label_encoder=False)
xgbc0.fit(x_train , y_train)

default_params = {}
gparams = xgbc0.get_params()

for key in gparams.keys():
    gp = gparams[key]
    default_params[key] = [gp]

clf0 = GridSearchCV(estimator=xgbc0, scoring='accuracy', param_grid=default_params, return_train_score=True, verbose=1, cv=3)
clf0.fit(x_train, y_train.values.ravel())

df = pd.DataFrame(clf0.cv_results_)

train_predictions = clf0.predict(x_train)
test_predictions = clf0.predict(x_test)

cfm_train = confusion_matrix(y_train, train_predictions)
cfm_test = confusion_matrix(y_test, test_predictions)

accs_train = accuracy_score(y_train, train_predictions)
accs_test = accuracy_score(y_test, test_predictions)

f1s_train_p1 = f1_score(y_train, train_predictions, pos_label=1)
f1s_train_p0 = f1_score(y_train, train_predictions, pos_label=0)
f1s_test_p1 = f1_score(y_test, test_predictions, pos_label=1)
f1s_test_p0 = f1_score(y_test, test_predictions, pos_label=0)

test_ras = roc_auc_score(y_test, clf0.predict_proba(x_test)[:,1])

bp = clf0.best_params_

results_dict['xgbc0'] = {'iterable_parameter': np.nan,
                         'classifier': deepcopy(clf0),
                         'cv_results': df.copy(),
                         'cfm_train': cfm_train,
                         'cfm_test': cfm_test,
                         'train_accuracy': accs_train,
                         'test_accuracy': accs_test,
                         'train F1-score label 1': f1s_train_p1,
                         'train F1-score label 0': f1s_train_p0,
                         'test F1-score label 1': f1s_test_p1,
                         'test F1-score label 0': f1s_test_p0,
                         'test roc auc score': test_ras,
                         'best_params': bp}

Fitting 3 folds for each of 1 candidates, totalling 3 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:665: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_folds = np.zeros(n_samples, dtype=np.int)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:437: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your cu

In [ ]:
bp

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'eval_metric': 'auc',
 'gamma': 0,
 'grow_policy': 'lossguide',
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 1,
 'tree_method': 'hist',
 'use_label_encoder': False,
 'verbosity': 1}

In [ ]:
params = deepcopy(default_params)

param_grid = {
  'gamma': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4, 200],
  'learning_rate': [0.01, 0.03, 0.06, 0.1, 0.15, 0.2, 0.25, 0.300000012, 0.4, 0.5, 0.6, 0.7],
  'max_depth': [5,6,7,8,9,10,11,12,13,14],
  'n_estimators': range(50, 220, 40),
  'reg_alpha': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
  'reg_lambda': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
  'min_child_weight': [1, 5, 10],
  'subsample': [0.6, 0.8, 1.0],
}

In [ ]:
print(param_grid)

{'gamma': [0, 0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6, 51.2, 102.4, 200], 'learning_rate': [0.01, 0.03, 0.06, 0.1, 0.15, 0.2, 0.25, 0.300000012, 0.4, 0.5, 0.6, 0.7], 'max_depth': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14], 'n_estimators': range(50, 220, 40), 'reg_alpha': [0, 0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6, 51.2, 102.4, 200], 'reg_lambda': [0, 0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6, 51.2, 102.4, 200], 'min_child_weight': [1, 5, 10], 'subsample': [0.6, 0.8, 1.0]}


In [ ]:
gcvj = np.cumsum([len(x) for x in param_grid.values()])[-1]

for i,grid_key in enumerate(param_grid.keys()):
    
    for param_key in params.keys():
        if param_key == grid_key:
            params[param_key] = param_grid[grid_key]
        else:
            try:
                param_value = [clf.best_params_[param_key]]
                params[param_key] = param_value
            except:
                param_value = [clf0.best_params_[param_key]]
                params[param_key] = param_value

    xgbc = xgb.XGBClassifier(**default_params)

    clf = GridSearchCV(estimator=xgbc, param_grid=params, scoring='accuracy', return_train_score=True, verbose=1, cv=3)
    clf.fit(x_train, y_train.values.ravel())

    df = pd.DataFrame(clf.cv_results_)

    train_predictions = clf.predict(x_train)
    test_predictions = clf.predict(x_test)

    cfm_train = confusion_matrix(y_train, train_predictions)
    cfm_test = confusion_matrix(y_test, test_predictions)

    accs_train = accuracy_score(y_train, train_predictions)
    accs_test = accuracy_score(y_test, test_predictions)

    f1s_train_p1 = f1_score(y_train, train_predictions, pos_label=1)
    f1s_train_p0 = f1_score(y_train, train_predictions, pos_label=0)
    f1s_test_p1 = f1_score(y_test, test_predictions, pos_label=1)
    f1s_test_p0 = f1_score(y_test, test_predictions, pos_label=0)

    test_ras = roc_auc_score(y_test, clf.predict_proba(x_test)[:,1])

    bp = clf.best_params_

    results_dict[f'xgbc{i+1}'] = {'iterable_parameter': grid_key,
                                  'classifier': deepcopy(clf),
                                  'cv_results': df.copy(),
                                  'cfm_train': cfm_train,
                                  'cfm_test': cfm_test,
                                  'train_accuracy': accs_train,
                                  'test_accuracy': accs_test,
                                  'train F1-score label 1': f1s_train_p1,
                                  'train F1-score label 0': f1s_train_p0,
                                  'test F1-score label 1': f1s_test_p1,
                                  'test F1-score label 0': f1s_test_p0,
                                  'test roc auc score': test_ras,
                                  'best_params': bp}

Fitting 3 folds for each of 13 candidates, totalling 39 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:665: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_folds = np.zeros(n_samples, dtype=np.int)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:437: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your cu

Fitting 3 folds for each of 12 candidates, totalling 36 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:665: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_folds = np.zeros(n_samples, dtype=np.int)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:437: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your cu

Fitting 3 folds for each of 10 candidates, totalling 30 fits


...

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:665: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_folds = np.zeros(n_samples, dtype=np.int)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:437: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your cu

Fitting 3 folds for each of 3 candidates, totalling 9 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:665: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_folds = np.zeros(n_samples, dtype=np.int)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:437: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your cu

Fitting 3 folds for each of 3 candidates, totalling 9 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:665: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_folds = np.zeros(n_samples, dtype=np.int)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:437: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your cu

In [ ]:
bp

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 0.6,
 'eval_metric': 'auc',
 'gamma': 0.2,
 'grow_policy': 'lossguide',
 'learning_rate': 0.2,
 'max_delta_step': 0,
 'max_depth': 9,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 210,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 12.8,
 'reg_lambda': 6.4,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 0.8,
 'tree_method': 'hist',
 'use_label_encoder': False,
 'verbosity': 1}

In [ ]:
#number of rows depend on number of iterations
nrows = len(results_dict.keys())

#standard group names for confusion matrices
group_names = ['True Neg','False Pos','False Neg','True Pos']

#creating figure
f, axes = plt.subplots(nrows,2,figsize=(18,8*nrows));

#iteratively plotting train/test accuracy scores and test confusion matrix
for i,ax in enumerate(axes):
    
    #current key of results dictionary
    ckey = list(results_dict.keys())[i] 
    
    #plotting scores for models other than the benchark model
    if ckey != 'xgbc0':
        x1 = results_dict[ckey]['cv_results'].loc[:,'mean_train_score']
        x2 = results_dict[ckey]['cv_results'].loc[:,'mean_test_score']
        
        ax[0].plot(x1, label='train scores', color='blue');
        ax[0].plot(x2, label='test scores', color='red');
        ax[0].set_title(f'Iteration #{i+1} results');
               
        ax[0].set_xticks(list(range(0,len([x[results_dict[ckey]['iterable_parameter']] for x in results_dict[ckey]['cv_results']['params']]))));
        ax[0].set_xticklabels(sorted([x[results_dict[ckey]['iterable_parameter']] for x in results_dict[ckey]['cv_results']['params']]));
    
        ax[0].grid('major');
        ax[0].legend();
        ax[0].set_xlabel(results_dict[ckey]['iterable_parameter'])
        ax[0].set_ylabel('mean score');
    
    #leaving scores plot blank for benchmark model
    else:
        ax[0].axis('off')
        ax[0].text(x=0.5, y=0.5, s='No iteration has been performed', fontsize=16, va='center', ha='center')
    
    #computing variables for specific confusion matrix
    group_counts = ["{0:0.0f}".format(value) for value in results_dict[ckey]['cfm_test'].flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in results_dict[ckey]['cfm_test'].flatten()/np.sum(results_dict[ckey]['cfm_test'])]
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    
plt.show();

KeyboardInterrupt: ignored

In [ ]:
print(results_dict['xgbc0']['train_accuracy'])
print(results_dict['xgbc0']['test_accuracy'])

0.8194311281559604
0.8095846645367412


In [ ]:
final_model = xgb.XGBClassifier(
  base_score= 0.5, 
  booster = 'gbtree', 
  colsample_bylevel= 1, 
  colsample_bynode = 1, 
  colsample_bytree=0.6, 
  eval_metric='auc', 
  gamma = 0.4, 
  grow_policy='lossguide',  
  learning_rate = 0.6, 
  max_delta_step=0, 
  max_depth = 12, 
  min_child_weight=1, 
  missing=None,  
  n_estimators = 145, 
  n_jobs=1, 
  nthread=None, 
  objective='binary:logistic',  
  reg_alpha = 12.8, 
  reg_lambda = 25.6, 
  random_state = 0, 
  scale_pos_weight=1, 
  seed=None, 
  silent=None, 
  subsample=0.8, 
  tree_method='hist', 
  use_label_encoder=False, 
  verbosity=1).fit(x_train, y_train)

In [ ]:
test_df = pd.read_csv('https://raw.githubusercontent.com/DiploDatos/AprendizajeSupervisado/master/Pr%C3%A1ctico/test.csv')

In [ ]:
PassengerId = test_df.PassengerId
PassengerId

0       415
1      8663
2      4524
3      8246
4      2935
       ... 
865    5996
866    5478
867    6760
868    1437
869    5171
Name: PassengerId, Length: 870, dtype: int64

In [ ]:
test_df.drop(['PassengerId','Name'], axis=1, inplace=True)
test_df.columns

Index(['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'],
      dtype='object')

In [ ]:
df_most_frequent_2 = test_df.copy()
categorical_cols = ['HomePlanet', 'CryoSleep', 'Cabin', 'VIP', 'Destination' ]
categorical_imputer = SimpleImputer(strategy='most_frequent')
df_most_frequent_2[categorical_cols] = categorical_imputer.fit_transform(df_most_frequent_2[categorical_cols])

df_most_frequent_2.isna().sum()

/usr/local/lib/python3.7/dist-packages/sklearn/impute.py:341: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  row_mask = np.logical_not(row_mask).astype(np.bool)
/usr/local/lib/python3.7/dist-packages/sklearn/impute.py:341: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  row_mask = np.logical_not(row_mask).astype(np.bool)
/usr/local/lib/python3.7/dist-

HomePlanet       0
CryoSleep        0
Cabin            0
Destination      0
Age             17
VIP              0
RoomService     27
FoodCourt       28
ShoppingMall    20
Spa             25
VRDeck          19
dtype: int64

In [ ]:
imputer_cols = ["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck" ,"RoomService"]
df_4 = df_most_frequent_2.copy()
imputer = SimpleImputer(strategy="median")
df_4[imputer_cols] = imputer.fit_transform(df_4[imputer_cols])

In [ ]:
df_4.isna().sum()

HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
dtype: int64

In [ ]:
df_4.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Europa,True,B/121/S,55 Cancri e,34.0,False,0.0,0.0,0.0,0.0,0.0
1,Earth,True,G/1489/S,PSO J318.5-22,13.0,False,0.0,0.0,0.0,0.0,0.0
2,Earth,False,G/776/P,TRAPPIST-1e,44.0,False,22.0,1.0,0.0,864.0,0.0
3,Earth,False,E/577/S,PSO J318.5-22,63.0,False,0.0,0.0,80.0,0.0,2322.0
4,Europa,False,B/117/S,55 Cancri e,10.0,False,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_5 = df_4.copy()
df_5[['Deck', 'Num', 'Side']] = df_5['Cabin'].str.split('/', expand=True)
df_5.drop(columns = 'Cabin', inplace = True)
df_5.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Num,Side
0,Europa,True,55 Cancri e,34.0,False,0.0,0.0,0.0,0.0,0.0,B,121,S
1,Earth,True,PSO J318.5-22,13.0,False,0.0,0.0,0.0,0.0,0.0,G,1489,S
2,Earth,False,TRAPPIST-1e,44.0,False,22.0,1.0,0.0,864.0,0.0,G,776,P
3,Earth,False,PSO J318.5-22,63.0,False,0.0,0.0,80.0,0.0,2322.0,E,577,S
4,Europa,False,55 Cancri e,10.0,False,0.0,0.0,0.0,0.0,0.0,B,117,S


In [ ]:
df_5 = df_4.copy()
df_5[['Deck', 'Num', 'Side']] = df_5['Cabin'].str.split('/', expand=True)
df_5.drop(columns = 'Cabin', inplace = True)
df_5.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Num,Side
0,Europa,True,55 Cancri e,34.0,False,0.0,0.0,0.0,0.0,0.0,B,121,S
1,Earth,True,PSO J318.5-22,13.0,False,0.0,0.0,0.0,0.0,0.0,G,1489,S
2,Earth,False,TRAPPIST-1e,44.0,False,22.0,1.0,0.0,864.0,0.0,G,776,P
3,Earth,False,PSO J318.5-22,63.0,False,0.0,0.0,80.0,0.0,2322.0,E,577,S
4,Europa,False,55 Cancri e,10.0,False,0.0,0.0,0.0,0.0,0.0,B,117,S


In [ ]:
categorical_cols_2 = ['HomePlanet', 'CryoSleep', 'VIP', 'Destination', 'Deck', 'Side']

for col in categorical_cols_2:
    df_5[col] = df_5[col].astype(str)
    df_5[col] = LabelEncoder().fit_transform(df_5[col])

In [ ]:
x_names = X.columns
X_test = StandardScaler().fit_transform(df_5)

X_test = pd.DataFrame(X_test, columns=x_names)
X_test

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Num,Side
0,0.506494,1.447786,-1.873124,0.389487,-0.136877,-0.385490,-0.310498,-0.274152,-0.309805,-0.250575,-1.873506,-0.950682,0.935414
1,-0.767060,1.447786,-0.634813,-1.050400,-0.136877,-0.385490,-0.310498,-0.274152,-0.309805,-0.250575,0.939663,1.704346,0.935414
2,-0.767060,-0.690710,0.603499,1.075147,-0.136877,-0.344213,-0.309830,-0.274152,0.441916,-0.250575,0.939663,0.320548,-1.069045
3,-0.767060,-0.690710,-0.634813,2.377902,-0.136877,-0.385490,-0.310498,-0.152063,-0.309805,1.277745,-0.185604,-0.065673,0.935414
4,0.506494,-0.690710,-1.873124,-1.256098,-0.136877,-0.385490,-0.310498,-0.274152,-0.309805,-0.250575,-1.873506,-0.958446,0.935414
...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,1.780047,-0.690710,0.603499,1.760807,-0.136877,0.882864,-0.310498,0.078379,0.019943,-0.250575,0.377029,1.166741,0.935414
866,-0.767060,1.447786,0.603499,2.035071,-0.136877,-0.385490,-0.310498,-0.274152,-0.309805,-0.250575,0.939663,0.652427,0.935414
867,-0.767060,1.447786,-0.634813,-0.707570,-0.136877,-0.385490,-0.310498,-0.274152,-0.309805,-0.250575,0.939663,1.042529,-1.069045
868,-0.767060,-0.690710,0.603499,-0.707570,-0.136877,1.049852,-0.310498,-0.254312,-0.309805,-0.250575,0.377029,-0.616863,0.935414


In [ ]:
test_id = PassengerId
test_pred = final_model.predict(X_test)
test_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1,

In [ ]:
submission = pd.DataFrame(list(zip(test_id, test_pred)), columns=["PassengerId", "Transported"])
submission.to_csv("sample_submission.csv", header=True, index=False)

In [ ]:
df_final = pd.read_csv("sample_submission.csv")
df_final

,PassengerId,Transported
0,415,1
1,8663,0
2,4524,0
3,8246,0
4,2935,1
...,...,...
865,5996,0
866,5478,1
867,6760,1
868,1437,0


In [ ]:
from google.colab import files
files.download('sample_submission.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>